In [1]:
!pip install rank_bm25

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trec-covid/trec-covid/queries.jsonl
/kaggle/input/trec-covid/trec-covid/corpus.jsonl
/kaggle/input/trec-covid/trec-covid/qrels/test.tsv


In [2]:
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask.dataframe as dd
import rank_bm25
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/trec-covid/trec-covid/queries.jsonl
/kaggle/input/trec-covid/trec-covid/corpus.jsonl
/kaggle/input/trec-covid/trec-covid/qrels/test.tsv


In [3]:
corpus = pd.read_json('/kaggle/input/trec-covid/trec-covid/corpus.jsonl',lines=True)
queries = pd.read_json('/kaggle/input/trec-covid/trec-covid/queries.jsonl',lines=True)
corpus=corpus.drop(columns='metadata')
queries=queries.drop(columns='metadata')


corpus_train = corpus.loc[0:128498]
corpus_test = corpus.loc[128499:]

# corpus_train = corpus.loc[0:1999]
# corpus_test = corpus.loc[1999:4000]

# queries_train = queries.loc[0:36]
# queries_test = queries.loc[37:]

queries_train = queries.loc[0:36]
queries_test = queries.loc[37:]

In [4]:
corpus_test.head(10)

,_id,title,text
128499,wwg6x1df,Clinical characteristics and prognosis in canc...,"• We describe the demographics, clinical chara..."
128500,3u01pn66,Humoral immunity to human metapneumovirus infe...,Human metapneumovirus (hMPV) is a significant ...
128501,aoiibz89,Respiratory Viral Infections after Hematopoiet...,This study was performed to characterize respi...
128502,5zi1i54p,Epitope Addition and Ablation via Manipulation...,"Despite the clinical relevance, dengue virus (..."
128503,7upcsmg4,COVID‐19 and dermatologic surgery: hazards of ...,
128504,rknygv7u,ASTHO at 75: Celebrating the Past and Preparin...,
128505,7ctp8bem,A multicenter surveillance of antimicrobial re...,Background Serratia marcescens is an important...
128506,f5qg7vcz,COVID-19 complicated by Acute Pulmonary Emboli...,Abstract A patient with Coronavirus Disease-20...
128507,nrhq7lqg,Bayesian Monitoring of Emerging Infectious Dis...,We define data analyses to monitor a change in...
128508,yql6gpd3,Tracking the Origin and Deciphering the Phylog...,"In 2010, new Chinese strains of porcine epidem..."


In [5]:
corpus_train['cleaned']= corpus_train['text'].apply(lambda x:x.lower())
corpus_test['cleaned']= corpus_test['text'].apply(lambda x:x.lower())

queries_train['cleaned']= queries_train['text'].apply(lambda x:x.lower())
queries_test['cleaned']= queries_test['text'].apply(lambda x:x.lower())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [6]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

In [7]:
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

corpus_train['cleaned']=corpus_train['cleaned'].apply(lambda x:expand_contractions(x))
corpus_test['cleaned']=corpus_test['cleaned'].apply(lambda x:expand_contractions(x))

queries_train['cleaned']=queries_train['cleaned'].apply(lambda x:expand_contractions(x))
queries_test['cleaned']=queries_test['cleaned'].apply(lambda x:expand_contractions(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [8]:
def clean_text(text):
    text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^a-z]',' ',text)
    return text
 
# Cleaning corpus using RegEx
corpus_train['cleaned']=corpus_train['cleaned'].apply(lambda x: clean_text(x))
corpus_test['cleaned']=corpus_test['cleaned'].apply(lambda x: clean_text(x))

queries_train['cleaned'] = queries_train['cleaned'].apply(lambda x: clean_text(x))
queries_test['cleaned'] = queries_test['cleaned'].apply(lambda x: clean_text(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [9]:
corpus_train['cleaned']=corpus_train['cleaned'].apply(lambda x: re.sub(' +',' ',x))
corpus_test['cleaned']=corpus_test['cleaned'].apply(lambda x: re.sub(' +',' ',x))

queries_test['cleaned']=queries_test['cleaned'].apply(lambda x: re.sub(' +',' ',x))
queries_train['cleaned']=queries_train['cleaned'].apply(lambda x: re.sub(' +',' ',x))




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [10]:
combined_training=pd.concat([corpus_train.rename(columns={'cleaned':'text'})['text'],\
                             queries_train.rename(columns={'cleaned':'text'})['text']])\
                             .sample(frac=1).reset_index(drop=True)

In [11]:
combined_training.head(10)

,text,text
0,,
1,BACKGROUND: We hereby report on our experience...,background we hereby report on our experience ...
2,Background Respiratory syncytial virus (RSV)- ...,background respiratory syncytial virus rsv and...
3,,
4,The current pandemic caused by severe acute re...,the current pandemic caused by severe acute re...
5,We address the satisfiability of systems of po...,we address the satisfiability of systems of po...
6,Diathermy has revolutionised modern surgery an...,diathermy has revolutionised modern surgery an...
7,,
8,A 73-year-old man was confirmed to have an inf...,a year old man was confirmed to have an influe...
9,,


In [12]:
# tokenized_corpus = [doc.split(" ")  for doc in combined_training]

# bm25 = BM25Okapi(tokenized_corpus)

In [13]:
from gensim.models import Word2Vec

# Creating data for the model training
train_data=[]
for i in combined_training:
    train_data.append(i.split())

# Training a word2vec model from the given data set
model = Word2Vec(train_data, min_count=2,window=5, sg=1,workers=4)
model.save("word2vec.model")

In [14]:
# Function returning vector reperesentation of a document
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(100)
    else:
        for tok in doc_tokens:
            if tok in model.wv.index_to_key:
                embeddings.append(model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(100))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

# Getting Word2Vec Vectors for Testing Corpus and Queries
corpus_test['vec']=corpus.text.apply(lambda x :get_embedding_w2v(x.split()))
queries_test['vec']=queries.text.apply(lambda x :get_embedding_w2v(x.split()))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [15]:
def ranking_ir(query):
  
  # pre-process Query
  query=query.lower()
  query=re.sub(' +',' ',query)

  # generating vector
  vector=get_embedding_w2v(query.split())

  # ranking documents
  documents = corpus_test
  documents['similarity']=corpus_test['vec'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  documents.sort_values(by='similarity',ascending=False,inplace=True)
  documents=documents.drop(columns='text')
  documents=documents.drop(columns='title')
  documents=documents.drop(columns='cleaned')
  documents=documents.drop(columns='vec')
  return documents.head(10)

In [16]:
ranking_ir(queries_test.text.iloc[0])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,_id,similarity
156790,j63cpzj2,0.989475
171304,vos341i1,0.989473
149197,nb4437qs,0.989473
167356,7hkowm2y,0.989116
143123,uvhc50qc,0.989064
148172,wk3fh9ke,0.989013
159605,bjhdy8mg,0.989002
136716,ixgz4e7k,0.988962
138418,qk0cyee2,0.988939
156641,8kktbmp2,0.988929


In [17]:
ranks = []
for query in queries_test.text:
    ranks.append(ranking_ir(query).values.tolist())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [18]:
import json
with open('output.txt', 'w') as filehandle:
    json.dump(ranks, filehandle)